In [1]:
# Celda 0.1 – Imports y configuración de paths correctos

import os
import random
from datetime import date, timedelta

import numpy as np
import pandas as pd
import yaml

# -------------------------------------------------------------------
# Rutas REALES según tu proyecto:
# notebooks/
#   └── 01_generate_stde_v3.ipynb
# data/
#     ├── oms/
#     ├── ontology/
#     └── synthetic/
# -------------------------------------------------------------------

RUTA_OMS = "../data/oms/99_oms_stde_v3.yaml"

# Catálogos en ontology/
PATH_ROLES     = "../data/ontology/10_catalogo_roles_v3.yaml"
PATH_TAREAS    = "../data/ontology/12_catalogo_tareas_v1.yaml"
PATH_AREAS     = "../data/ontology/13_catalogo_areas_operacionales_v1.yaml"
PATH_RIESGOS   = "../data/ontology/01_catalogo_riesgos_v8.yaml"
PATH_CONTROLES = "../data/ontology/02_catalogo_controles_v6.yaml"

# Modelo proactivo en oms/
PATH_PROACTIVO = "../data/oms/stde_proactivo_semanal_v4_4.csv"

# Directorio de salida (synthetic/)
OUTPUT_DIR = "../data/synthetic/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Configuración de aleatoriedad
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)



In [2]:
# Celda 0.2 – Cargar OMS v3

def cargar_yaml(path):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

oms = cargar_yaml(RUTA_OMS)

# Accesos rápidos a secciones relevantes de la OMS v3
meta = oms.get("meta", {})
stde_cfg = oms.get("stde", {})
fdo_cfg = oms.get("fdo", {})
escenario_cfg = oms.get("escenario_operacion", {})
mapeos_operacionales = oms.get("mapeos_operacionales", {})
proactivo_cfg = oms.get("proactivo", {})

lunes_critico_cfg = escenario_cfg.get("lunes_critico", {})
eventos_lunes_critico = lunes_critico_cfg.get("eventos_sinteticos", [])

print("Versión OMS:", meta.get("version"))
print("Fecha inicio escenario:", escenario_cfg.get("fecha_inicio"))
print("Número de semanas:", escenario_cfg.get("numero_semanas"))
print("Eventos del lunes crítico:", len(eventos_lunes_critico))


Versión OMS: None
Fecha inicio escenario: 2024-11-11
Número de semanas: 12
Eventos del lunes crítico: 2


In [3]:
# Celda 0.3 – Cargar catálogos de ontología (roles, tareas, áreas, riesgos, controles)

# Celda 0.3 – Cargar catálogos oficiales (función única)

def cargar_catalogo(path):
    """
    Carga catálogos YAML que pueden estar en dos formatos:
    1) Una lista directamente en la raíz.
    2) Un diccionario cuya primera clave contiene una lista.

    Retorna siempre un DataFrame.
    """
    data = cargar_yaml(path)

    # Caso 1: El archivo es directamente una lista
    if isinstance(data, list):
        return pd.DataFrame(data)

    # Caso 2: El archivo es un diccionario con una lista dentro
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(v, list):
                return pd.DataFrame(v)

    raise ValueError(f"Formato no reconocido en catálogo: {path}")


# ======================
# CARGA DE CATÁLOGOS
# ======================

df_roles     = cargar_catalogo(PATH_ROLES)
df_tareas    = cargar_catalogo(PATH_TAREAS)
df_areas     = cargar_catalogo(PATH_AREAS)
df_riesgos   = cargar_catalogo(PATH_RIESGOS)
df_controles = cargar_catalogo(PATH_CONTROLES)

print("Catálogos cargados correctamente:")
print("Roles:", df_roles.shape)
print("Tareas:", df_tareas.shape)
print("Áreas:", df_areas.shape)
print("Riesgos:", df_riesgos.shape)
print("Controles:", df_controles.shape)




Catálogos cargados correctamente:
Roles: (45, 7)
Tareas: (96, 9)
Áreas: (5, 7)
Riesgos: (3, 16)
Controles: (48, 14)


In [4]:
# Celda 1.1 – Generar calendario diario del escenario STDE (12 semanas) + marca de lunes crítico

def generar_calendario(escenario_cfg):
    fecha_inicio_str = escenario_cfg["fecha_inicio"]
    numero_semanas = escenario_cfg["numero_semanas"]
    fecha_lunes_critico_str = escenario_cfg["lunes_critico"]["fecha"]

    fecha_inicio = date.fromisoformat(fecha_inicio_str)
    dias_totales = numero_semanas * 7  # 12 semanas → 84 días (sin lunes crítico)
    fecha_lunes_critico = date.fromisoformat(fecha_lunes_critico_str)

    registros = []
    for i in range(dias_totales):
        fecha = fecha_inicio + timedelta(days=i)
        semana = (i // 7) + 1  # semana 1..12
        dia_semana = fecha.weekday()  # 0 = lunes, 6 = domingo

        registros.append(
            {
                "id_dia": i + 1,
                "fecha": fecha,
                "semana": semana,
                "dia_semana": dia_semana,
                "es_lunes_critico": (fecha == fecha_lunes_critico),
            }
        )

    calendario = pd.DataFrame(registros)
    return calendario

calendario = generar_calendario(escenario_cfg)
print("Días calendario:", len(calendario))
calendario.head()


Días calendario: 84


,id_dia,fecha,semana,dia_semana,es_lunes_critico
0,1,2024-11-11,1,0,False
1,2,2024-11-12,1,1,False
2,3,2024-11-13,1,2,False
3,4,2024-11-14,1,3,False
4,5,2024-11-15,1,4,False


In [5]:
# Celda 1.2 – Generar FDO diario (7 factores) con patrones suaves

fdo_factores = fdo_cfg.get("factores", [])

# Creamos un índice de factores por id para referencia si se requiere después
FDO_IDS = [f["id"] for f in fdo_factores]

def generar_fdo_diario(calendario):
    registros = []
    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        # Base de variación semanal: degradación leve hacia semana 12
        # Normalizamos semana 1..12 a 0..1
        t = (semana - 1) / (calendario["semana"].max() - 1)

        # Definimos patrones simplificados coherentes con la narrativa:
        # - Producción: crece con el tiempo (más presión)
        # - Backlog: crece (mantenimiento atrasado)
        # - Congestión: crece con producción
        # - Fatiga: crece, modulada por día de la semana (más alta al final de la semana)
        # - Clima: variación aleatoria suave
        # - Dotación: algo decreciente (baja levemente)
        # - Variabilidad: aumenta moderadamente
        dow = row["dia_semana"]  # 0 lunes, 6 domingo

        fdo_produccion = np.clip(0.3 + 0.6 * t + np.random.normal(0, 0.05), 0.0, 1.0)
        fdo_backlog = np.clip(0.2 + 0.7 * t + np.random.normal(0, 0.05), 0.0, 1.0)
        fdo_congestion = np.clip(0.25 + 0.6 * t + np.random.normal(0, 0.05), 0.0, 1.0)

        # Fatiga: más alta jueves-viernes (3,4)
        fatiga_semana = 0.2 + 0.6 * t
        ajuste_dow = 0.1 if dow in (3, 4) else 0.0
        fdo_fatiga = np.clip(fatiga_semana + ajuste_dow + np.random.normal(0, 0.05), 0.0, 1.0)

        # Clima: ruido con tendencia leve
        fdo_clima = np.clip(0.4 + 0.2 * np.sin(2 * np.pi * t) + np.random.normal(0, 0.1), 0.0, 1.0)

        # Dotación: leve caída con el tiempo (renuncias, licencias, etc.)
        fdo_dotacion = np.clip(0.9 - 0.3 * t + np.random.normal(0, 0.05), 0.0, 1.0)

        # Variabilidad: aumenta hacia el final (más cambios de plan)
        fdo_variabilidad = np.clip(0.2 + 0.6 * t + np.random.normal(0, 0.05), 0.0, 1.0)

        registros.append(
            {
                "id_dia": id_dia,
                "fecha": fecha,
                "semana": semana,
                "fdo_produccion": fdo_produccion,
                "fdo_backlog": fdo_backlog,
                "fdo_congestion": fdo_congestion,
                "fdo_fatiga": fdo_fatiga,
                "fdo_clima": fdo_clima,
                "fdo_dotacion": fdo_dotacion,
                "fdo_variabilidad": fdo_variabilidad,
            }
        )

    df_fdo = pd.DataFrame(registros)
    return df_fdo

df_fdo_diario = generar_fdo_diario(calendario)
print("FDO diario generado:", df_fdo_diario.shape)
df_fdo_diario.head()
df_fdo = df_fdo_diario.copy()



FDO diario generado: (84, 10)


In [6]:
# Celda 2.1 – Generar trayectorias diarias de criticidad para R01, R02, R03

RIESGOS_FOCO = ["R01", "R02", "R03"]

def generar_trayectorias_riesgo(calendario):
    registros = []
    max_semana = calendario["semana"].max()

    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        # Normalización temporal 0..1 sobre las 12 semanas
        t = (semana - 1) / (max_semana - 1)

        # -------------------------------
        # R02 – Degradación acumulada fuerte
        # -------------------------------
        base_R02 = 0.35 + 0.5 * t  # crece fuertemente hacia el final
        ruido_R02 = np.random.normal(0, 0.03)
        criticidad_R02 = np.clip(base_R02 + ruido_R02, 0.0, 1.0)

        # -------------------------------
        # R01 – Controlado pero con señales débiles
        # -------------------------------
        base_R01 = 0.25 + 0.25 * t  # leve pendiente
        ruido_R01 = np.random.normal(0, 0.03)
        criticidad_R01 = np.clip(base_R01 + ruido_R01, 0.0, 1.0)

        # -------------------------------
        # R03 – Estable con variaciones leves
        # -------------------------------
        base_R03 = 0.18  # valor central estable
        ruido_R03 = np.random.uniform(-0.03, 0.03)
        pico_R03 = 0.02 if random.random() < 0.05 else 0.0
        criticidad_R03 = np.clip(base_R03 + ruido_R03 + pico_R03, 0.0, 1.0)


        registros.append(
            {
                "id_dia": id_dia,
                "fecha": fecha,
                "semana": semana,
                "criticidad_R01": criticidad_R01,
                "criticidad_R02": criticidad_R02,
                "criticidad_R03": criticidad_R03,
            }
        )

    df_tray = pd.DataFrame(registros)
    return df_tray

df_trayectorias = generar_trayectorias_riesgo(calendario)
print("Trayectorias generadas:", df_trayectorias.shape)
df_trayectorias.head()


Trayectorias generadas: (84, 6)


,id_dia,fecha,semana,criticidad_R01,criticidad_R02,criticidad_R03
0,1,2024-11-11,1,0.224317,0.374910,0.152012
1,2,2024-11-12,1,0.262061,0.395293,0.222606
2,3,2024-11-13,1,0.234700,0.381126,0.156638
3,4,2024-11-14,1,0.236661,0.388609,0.158995
4,5,2024-11-15,1,0.222335,0.372710,0.193202


In [7]:
df_trayectorias.tail(10)


,id_dia,fecha,semana,criticidad_R01,criticidad_R02,criticidad_R03
74,75,2025-01-24,11,0.508867,0.772114,0.205464
75,76,2025-01-25,11,0.474860,0.838418,0.200673
76,77,2025-01-26,11,0.461791,0.832694,0.152220
77,78,2025-01-27,12,0.515031,0.807845,0.157234
78,79,2025-01-28,12,0.485638,0.856664,0.170392
79,80,2025-01-29,12,0.504050,0.842169,0.168917
80,81,2025-01-30,12,0.505897,0.893409,0.169125
81,82,2025-01-31,12,0.508491,0.876033,0.186952
82,83,2025-02-01,12,0.531964,0.852469,0.206344
83,84,2025-02-02,12,0.479497,0.886695,0.174116


In [8]:
# Celda 2.2 – Trayectoria ponderada global por día y agregados por semana

PESOS_RIESGO_GLOBAL = {
    "R01": 0.25,
    "R02": 0.5,
    "R03": 0.25,
}

def agregar_trayectoria_global(df_tray):
    df = df_tray.copy()

    df["criticidad_global"] = (
        df["criticidad_R01"] * PESOS_RIESGO_GLOBAL["R01"]
        + df["criticidad_R02"] * PESOS_RIESGO_GLOBAL["R02"]
        + df["criticidad_R03"] * PESOS_RIESGO_GLOBAL["R03"]
    )

    # Agregados semanales para facilitar gráficos en la demo (medias por semana)
    df_semana = (
        df.groupby("semana", as_index=False)[
            [
                "criticidad_R01",
                "criticidad_R02",
                "criticidad_R03",
                "criticidad_global",
            ]
        ]
        .mean()
        .rename(
            columns={
                "criticidad_R01": "criticidad_R01_media",
                "criticidad_R02": "criticidad_R02_media",
                "criticidad_R03": "criticidad_R03_media",
                "criticidad_global": "criticidad_global_media",
            }
        )
    )

    return df, df_semana

df_trayectorias, df_trayectorias_semana = agregar_trayectoria_global(df_trayectorias)

print("df_trayectorias:", df_trayectorias.shape)
print("df_trayectorias_semana:", df_trayectorias_semana.shape)
df_trayectorias_semana.head()


df_trayectorias: (84, 7)
df_trayectorias_semana: (12, 5)


,semana,criticidad_R01_media,criticidad_R02_media,criticidad_R03_media,criticidad_global_media
0,1,0.238999,0.375859,0.175273,0.291497
1,2,0.298600,0.410016,0.184553,0.325796
2,3,0.277549,0.437501,0.193956,0.336627
3,4,0.321048,0.491794,0.170725,0.368840
4,5,0.319023,0.540638,0.175150,0.393862


In [9]:
# ------------------------------------------------------------
# Celda 3.1a — Funciones base de probabilidad (PD, NMS, IMEN)
# Versión FINAL calibrada (mining-realistic) para STDE v3
# ------------------------------------------------------------

def base_PD(criticidad, fdo_row):
    """
    HAZARD (Peligro Detectado)
    Debe ser el evento más frecuente en minería (~70–80% del total).
    """
    p = (
        0.18                       # antes 0.10
        + 0.12 * criticidad        # antes 0.05
    )

    p *= (1 + 0.20 * fdo_row["fdo_congestion"])

    return min(p, 0.40)            # antes 0.25



def base_NM(criticidad, fdo_row):
    """
    NMS (Near Miss)
    Frecuencia intermedia (ideal 15–25%), solo sube con criticidad alta.
    """
    p = (
        0.04                      # base antes 0.02 → duplicamos
        + 0.06 * criticidad       # antes 0.04 → más sensibilidad
    )

    # fdo_clima sube efecto realista
    p *= (1 + 0.10 * fdo_row["fdo_clima"])

    return min(p, 0.12)           # antes 0.07 → límite mayor
          # hard cap 7%


def base_IM(criticidad, fdo_row):
    """
    IMEN (Incidente Menor)
    RARO: 0.3–1.5% normalmente.
    """
    p = (
        0.003                      # base 0.3%
        + 0.01 * criticidad        # criticidad aporta hasta 1%
    )

    p *= (1 + 0.05 * fdo_row["fdo_fatiga"])

    return min(p, 0.02)            # límite duro 2%


print("Funciones PD / NMS / IM recalibradas (versión final).")


Funciones PD / NMS / IM recalibradas (versión final).


In [10]:
# Celda 3.1b – Versión corregida de modular_por_fdo (redefine solo esta función)

def modular_por_fdo(prob, fdo_row):
    """
    Ajusta la probabilidad con los FDO diarios.
    Reglas STDE v3:
      - producción, congestión y fatiga aumentan prob. de eventos
      - dotación baja → aumenta riesgo
      - clima extremo → altera probabilidad
    """
    factor = (
        1.0
        + 0.30 * fdo_row["fdo_produccion"]
        + 0.20 * fdo_row["fdo_congestion"]
        + 0.20 * fdo_row["fdo_fatiga"]
        + 0.10 * (1.0 - fdo_row["fdo_dotacion"])   # menos dotación → más riesgo
        + 0.10 * abs(fdo_row["fdo_clima"] - 0.5)   # clima muy bueno o muy malo altera el riesgo
    )

    prob_mod = np.clip(prob * factor, 0.0, 0.99)
    return prob_mod


In [11]:
# --------------------------------------------------------------------
# 3.2a Preprocesar roles y tareas agrupados por área (usando 'area_operacional')
# --------------------------------------------------------------------

# Validar columnas reales
assert "area_operacional" in df_roles.columns, "El catálogo de roles no contiene 'area_operacional'"
assert "area_operacional" in df_tareas.columns, "El catálogo de tareas no contiene 'area_operacional'"

# Crear diccionarios área → lista de roles / tareas
roles_por_area = (
    df_roles
    .groupby("area_operacional")["id"]
    .apply(list)
    .to_dict()
)

tareas_por_area = (
    df_tareas
    .groupby("area_operacional")["id"]
    .apply(list)
    .to_dict()
)

print("Roles por área:", {k: len(v) for k,v in roles_por_area.items()})
print("Tareas por área:", {k: len(v) for k,v in tareas_por_area.items()})


Roles por área: {'CHP': 9, 'MRA': 8, 'PLC': 12, 'STM': 8, 'TME': 8}
Tareas por área: {'CHP': 20, 'MRA': 22, 'PLC': 19, 'STM': 19, 'TME': 16}


In [12]:
# ------------------------------------------------------------
# Celda 3.2 — Generar eventos diarios (HAZARD / NMS / IMEN)
# ------------------------------------------------------------

def generar_eventos_v3(
    calendario, df_trayectorias, df_fdo, oms,
    roles_por_area, tareas_por_area
):
    eventos = []

    # Extraer mapeo riesgo_por_area desde OMS v3
    riesgo_por_area = oms.get("mapeos_operacionales", {}).get("riesgo_por_area", {})

    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        # FDO y trayectoria para ese día
        fdo_row = df_fdo[df_fdo["id_dia"] == id_dia].iloc[0]
        tra_row = df_trayectorias[df_trayectorias["id_dia"] == id_dia].iloc[0]

        # Recorrer cada área definida en la OMS
        for area_id, riesgos_area in riesgo_por_area.items():

            # 1) Elegir riesgo dominante del área según pesos
            riesgos = [x["riesgo_id"] for x in riesgos_area]
            pesos = [x["peso_relativo"] for x in riesgos_area]
            riesgo_elegido = random.choices(riesgos, weights=pesos, k=1)[0]

            # 2) Obtener criticidad del riesgo elegido
            criticidad = float(tra_row[f"criticidad_{riesgo_elegido}"])

            # 3) Calcular probabilidad de evento
            p_pd = base_PD(criticidad, fdo_row)
            p_nm = base_NM(criticidad, fdo_row)
            p_im = base_IM(criticidad, fdo_row)

            # 4) Selección del tipo narrativo
            evento_tipo = None
            r = random.random()

            # Orden lógico PD → NMS → IMEN
            # Probabilidad total
            p_total = p_pd + p_nm + p_im
            r = random.random()

            evento_tipo = None
            if r < p_pd:
                evento_tipo = "HAZARD"
            elif r < p_pd + p_nm:
                evento_tipo = "NMS"
            elif r < p_pd + p_nm + p_im:
                evento_tipo = "IMEN"


            # 5) Si no ocurrió evento, pasar al siguiente área
            if not evento_tipo:
                continue

            # 6) Seleccionar rol/tarea válidos del área
            roles_area = roles_por_area.get(area_id, [])
            tareas_area = tareas_por_area.get(area_id, [])

            if not roles_area or not tareas_area:
                continue  # Seguridad

            rol_id = random.choice(roles_area)
            tarea_id = random.choice(tareas_area)

            # 7) Registrar evento
            eventos.append({
                "id_dia": id_dia,
                "fecha": fecha,
                "semana": semana,
                "id_area": area_id,
                "riesgo_id": riesgo_elegido,
                "tipo_evento": evento_tipo,
                "criticidad": criticidad,
                "rol_id": rol_id,
                "tarea_id": tarea_id,
            })

    return pd.DataFrame(eventos)


# Ejecutar
df_eventos = generar_eventos_v3(
    calendario, df_trayectorias, df_fdo, oms,
    roles_por_area, tareas_por_area
)

print("Eventos generados:", df_eventos.shape)
df_eventos.head()


Eventos generados: (155, 9)


,id_dia,fecha,semana,id_area,riesgo_id,tipo_evento,criticidad,rol_id,tarea_id
0,1,2024-11-11,1,CHP,R02,HAZARD,0.374910,CHP_ROL_02,CHP_TA_005
1,2,2024-11-12,1,STM,R03,NMS,0.222606,STM_ROL_03,STM_TA_017
2,3,2024-11-13,1,MRA,R02,HAZARD,0.381126,MRA_ROL_03,MRA_TA_018
3,3,2024-11-13,1,TME,R03,HAZARD,0.156638,TME_ROL_06,TME_TA_010
4,4,2024-11-14,1,MRA,R03,HAZARD,0.158995,MRA_ROL_08,MRA_TA_018


In [13]:
df_eventos["tipo_evento"].value_counts()


tipo_evento
HAZARD    113
NMS        37
IMEN        5
Name: count, dtype: int64

In [14]:
df_eventos["riesgo_id"].value_counts()


riesgo_id
R02    85
R03    35
R01    35
Name: count, dtype: int64

In [15]:
df_eventos.groupby("semana")["tipo_evento"].count()


semana
1      8
2     12
3     13
4     14
5     12
6     10
7     12
8     13
9     18
10    15
11    16
12    12
Name: tipo_evento, dtype: int64

In [16]:
# ------------------------------------------------------------
# Celda 3.3 — Inyección del lunes crítico (versión C.3 — área fija MRA)
# ------------------------------------------------------------

def inyectar_lunes_critico(df_eventos, df_trayectorias, oms,
                           roles_por_area, tareas_por_area):
    """
    Inyecta el lunes crítico como un evento compuesto:
      - LC_EVT_00 (evento maestro)
      - LC_EVT_01 (NMS R01)
      - LC_EVT_02 (NMS R02)

    NOTA:
    Para mantener la coherencia narrativa, el lunes crítico
    siempre ocurre en el área MRA, independientemente del
    mapeo operacional general.
    """

    # Fecha del LC definida en OMS v3
    fecha_lc = pd.to_datetime(
        oms["escenario_operacion"]["lunes_critico"]["fecha"]
    ).date()

    semana_lc = 13
    id_dia_lc = -1

    # Criticidad real (último día semana 12)
    def criticidad_real(riesgo_id):
        try:
            fila = df_trayectorias[df_trayectorias["semana"] == 12].iloc[-1]
            return float(fila[f"criticidad_{riesgo_id}"])
        except:
            return 0.8

    # Forzar área = MRA
    area_lc = "MRA"

    # Selección de rol/tarea solo para MRA
    def elegir_rol_tarea_MRA():
        roles = roles_por_area.get(area_lc, [])
        tareas = tareas_por_area.get(area_lc, [])
        if not roles or not tareas:
            return None, None
        return random.choice(roles), random.choice(tareas)

    rol_R01, tarea_R01 = elegir_rol_tarea_MRA()
    rol_R02, tarea_R02 = elegir_rol_tarea_MRA()

    registros = []

    # ------------------------------------------------------------
    # Evento Maestro (compuesto)
    # ------------------------------------------------------------
    registros.append({
        "id_evento": "LC_EVT_00",
        "id_evento_compuesto": None,
        "id_dia": id_dia_lc,
        "fecha": fecha_lc,
        "semana": semana_lc,

        "id_area": area_lc,
        "rol_id": None,
        "tarea_id": None,

        "riesgo_id": None,
        "tipo_evento": "NMS_COMPUESTO",
        "criticidad": max(criticidad_real("R01"), criticidad_real("R02")),
        "es_lunes_critico": True,

        "descripcion_evento": (
            "Durante una inspección rutinaria en un andamio móvil, un trabajador perdió "
            "el equilibrio y simultáneamente se le cayó una llave desde altura. "
            "El arnés detuvo la caída y la herramienta no impactó a nadie."
        ),
        "evento_principal_ocurrido": (
            "Pérdida de equilibrio en andamio que activa caída detenida (R01) "
            "y caída de objeto (R02)."
        )
    })

    # ------------------------------------------------------------
    # Sub-evento R01 (Caída detenida)
    # ------------------------------------------------------------
    registros.append({
        "id_evento": "LC_EVT_01",
        "id_evento_compuesto": "LC_EVT_00",
        "id_dia": id_dia_lc,
        "fecha": fecha_lc,
        "semana": semana_lc,

        "id_area": area_lc,
        "rol_id": rol_R01,
        "tarea_id": tarea_R01,

        "riesgo_id": "R01",
        "tipo_evento": "NMS",
        "criticidad": criticidad_real("R01"),
        "es_lunes_critico": True,

        "descripcion_evento": (
            "Caída detenida por arnés al perder equilibrio en el andamio."
        ),
        "evento_principal_ocurrido": "Caída detenida."
    })

    # ------------------------------------------------------------
    # Sub-evento R02 (Caída de herramienta)
    # ------------------------------------------------------------
    registros.append({
        "id_evento": "LC_EVT_02",
        "id_evento_compuesto": "LC_EVT_00",
        "id_dia": id_dia_lc,
        "fecha": fecha_lc,
        "semana": semana_lc,

        "id_area": area_lc,
        "rol_id": rol_R02,
        "tarea_id": tarea_R02,

        "riesgo_id": "R02",
        "tipo_evento": "NMS",
        "criticidad": criticidad_real("R02"),
        "es_lunes_critico": True,

        "descripcion_evento": (
            "Caída de herramienta desde el andamio sin impacto en personas."
        ),
        "evento_principal_ocurrido": "Caída de objeto."
    })

    df_lc = pd.DataFrame(registros)

    df_final = (
        pd.concat([df_eventos, df_lc], ignore_index=True)
        .sort_values(["fecha", "id_area"])
        .reset_index(drop=True)
    )

    print("→ Lunes crítico inyectado correctamente (área=MRA).")
    return df_final


# Reinyectar LC
df_eventos = inyectar_lunes_critico(
    df_eventos, df_trayectorias, oms,
    roles_por_area, tareas_por_area
)

df_eventos[df_eventos["es_lunes_critico"] == True]


→ Lunes crítico inyectado correctamente (área=MRA).


,id_dia,fecha,semana,id_area,riesgo_id,tipo_evento,criticidad,rol_id,tarea_id,id_evento,id_evento_compuesto,es_lunes_critico,descripcion_evento,evento_principal_ocurrido
155,-1,2025-02-03,13,MRA,None,NMS_COMPUESTO,0.886695,None,None,LC_EVT_00,None,True,Durante una inspección rutinaria en un andamio...,Pérdida de equilibrio en andamio que activa ca...
156,-1,2025-02-03,13,MRA,R01,NMS,0.479497,MRA_ROL_08,MRA_TA_003,LC_EVT_01,LC_EVT_00,True,Caída detenida por arnés al perder equilibrio ...,Caída detenida.
157,-1,2025-02-03,13,MRA,R02,NMS,0.886695,MRA_ROL_01,MRA_TA_006,LC_EVT_02,LC_EVT_00,True,Caída de herramienta desde el andamio sin impa...,Caída de objeto.


In [17]:
# ------------------------------------------------------------
# Celda 4.1 — Observaciones OPG± (ruido operacional por área)
# ------------------------------------------------------------

def generar_observaciones_opg(calendario, df_fdo, roles_por_area, tareas_por_area):
    registros = []

    # Reglas de carga por área
    OPG_AREA = {
        "MRA": (2, 4),   # Rango por día
        "CHP": (1, 3),
        "PLC": (1, 3),
        "STM": (1, 3),
        "TME": (1, 3),
    }

    for _, row in calendario.iterrows():
        fecha = row["fecha"]
        semana = row["semana"]
        id_dia = row["id_dia"]

        # FDO del día
        fdo_row = df_fdo[df_fdo["id_dia"] == id_dia].iloc[0]
        fatiga = fdo_row["fdo_fatiga"]
        congestion = fdo_row["fdo_congestion"]

        for area_id, (lo, hi) in OPG_AREA.items():

            # Cantidad de observaciones del día en esa área
            n_opg = random.randint(lo, hi)

            for _ in range(n_opg):

                # Selección de roles/tareas válidos
                roles_area = roles_por_area.get(area_id, [])
                tareas_area = tareas_por_area.get(area_id, [])

                if not roles_area or not tareas_area:
                    continue

                rol_id = random.choice(roles_area)
                tarea_id = random.choice(tareas_area)

                # Estado OPG+ / OPG−
                # Congestión y fatiga aumentan probabilidad de OPG−
                p_neg = 0.03 + 0.10 * congestion + 0.07 * fatiga
                p_neg = min(p_neg, 0.20)  # límite duro: 20%

                estado = "OPG-" if random.random() < p_neg else "OPG+"

                registros.append({
                    "id_observacion": None,  # se completa luego en consolidación
                    "fecha": fecha,
                    "semana": semana,
                    "id_area": area_id,
                    "rol_observador_id": rol_id,
                    "rol_observado_id": rol_id,  # simplificación STDE v3
                    "tarea_id": tarea_id,
                    "tipo_observacion": "OPG",
                    "estado": estado,
                    "riesgo_id": None,
                    "is_control_critico": False,
                    "control_critico_id": None,
                })

    return pd.DataFrame(registros)


# Ejecutar OPG±
df_opg = generar_observaciones_opg(
    calendario, df_fdo, roles_por_area, tareas_por_area
)

print("OPG generadas:", df_opg.shape)
df_opg.head()


OPG generadas: (933, 12)


,id_observacion,fecha,semana,id_area,rol_observador_id,rol_observado_id,tarea_id,tipo_observacion,estado,riesgo_id,is_control_critico,control_critico_id
0,None,2024-11-11,1,MRA,MRA_ROL_07,MRA_ROL_07,MRA_TA_016,OPG,OPG+,None,False,None
1,None,2024-11-11,1,MRA,MRA_ROL_06,MRA_ROL_06,MRA_TA_020,OPG,OPG+,None,False,None
2,None,2024-11-11,1,MRA,MRA_ROL_06,MRA_ROL_06,MRA_TA_012,OPG,OPG+,None,False,None
3,None,2024-11-11,1,CHP,CHP_ROL_06,CHP_ROL_06,CHP_TA_017,OPG,OPG+,None,False,None
4,None,2024-11-11,1,PLC,PLC_ROL_04,PLC_ROL_04,PLC_TA_015,OPG,OPG+,None,False,None


In [18]:
# ------------------------------------------------------------
# Celda 4.2 – OCC± planificadas/espontáneas según ranking proactivo
#   - Total objetivo ≈ 200 OCC en 12 semanas (~16/semana)
#   - Solo riesgos R01–R03
#   - Distribución semanal:
#       50% riesgo top entre R01–R03
#       30% segundo
#       20% tercero
#   - NO dependen de eventos, solo de:
#       - ranking semanal (modelo proactivo)
#       - mapeos_operacionales.riesgo_por_area (OMS v3)
#       - catálogos de roles/tareas/controles
# ------------------------------------------------------------

def generar_occ_planificadas(
    calendario,
    oms,
    roles_por_area,
    tareas_por_area,
    df_controles,
    path_proactivo=PATH_PROACTIVO,
    occ_totales_por_semana=16,
):
    """
    Genera OCC± para 12 semanas, usando el ranking proactivo v4.4:
    - Se lee el CSV del modelo proactivo (stde_proactivo_semanal_v4_4.csv)
    - Para cada semana se determina el orden relativo de R01, R02, R03.
    - Se asignan ~16 OCC semanales distribuidas 50/30/20 según ese orden.
    - Las OCC se distribuyen en días y áreas usando mapeos_operacionales.
    """

    # ------------------------------
    # 1) Cargar ranking proactivo
    # ------------------------------
    df_pro = pd.read_csv(path_proactivo)

    # Normalizar nombres de columnas si fuese necesario
    # Se espera: "semana_id", "riesgo_id", "score_proactivo", "rank_proactivo"
    cols_lower = {c.lower(): c for c in df_pro.columns}
    if "semana_id" not in df_pro.columns and "semana_id" in cols_lower:
        df_pro.rename(columns={cols_lower["semana_id"]: "semana_id"}, inplace=True)
    if "riesgo_id" not in df_pro.columns and "riesgo_id" in cols_lower:
        df_pro.rename(columns={cols_lower["riesgo_id"]: "riesgo_id"}, inplace=True)
    if "rank_proactivo" not in df_pro.columns and "rank_proactivo" in cols_lower:
        df_pro.rename(columns={cols_lower["rank_proactivo"]: "rank_proactivo"}, inplace=True)

    # Nos quedamos solo con R01–R03
    df_pro_3 = df_pro[df_pro["riesgo_id"].isin(["R01", "R02", "R03"])].copy()

    # ------------------------------
    # 2) Mapeos de áreas por riesgo (OMS v3)
    # ------------------------------
    riesgo_por_area = oms.get("mapeos_operacionales", {}).get("riesgo_por_area", {})

    def elegir_area_para_riesgo(riesgo_id):
        """Elige un área para el riesgo dado, usando los pesos de mapeos_operacionales."""
        areas = []
        pesos = []

        for area_id, lista_riesgos in riesgo_por_area.items():
            for item in lista_riesgos:
                if item["riesgo_id"] == riesgo_id:
                    areas.append(area_id)
                    pesos.append(item["peso_relativo"])
        if not areas:
            return None
        return random.choices(areas, weights=pesos, k=1)[0]

    # ------------------------------
    # 3) Controles críticos por riesgo
    # ------------------------------
    # El catálogo de controles tiene "riesgo_asociado" y "id"
    controles_por_riesgo = (
        df_controles.groupby("riesgo_asociado")["id"]
        .apply(list).to_dict()
    )

    def elegir_control_critico(riesgo_id):
        ids = controles_por_riesgo.get(riesgo_id, [])
        if not ids:
            return None
        return random.choice(ids)

    # ------------------------------
    # 4) Bucle por semana (1–12)
    # ------------------------------
    registros = []

    semanas_unicas = sorted(calendario["semana"].unique())
    # asumimos 12 semanas numeradas 1..12
    for semana in semanas_unicas:
        # Ranking para esa semana (solo R01–R03)
        df_sem = df_pro_3[df_pro_3["semana_id"] == semana]

        if df_sem.empty:
            continue

        # Ordenar por rank_proactivo (1 = más crítico)
        df_sem_ordenado = df_sem.sort_values("rank_proactivo")
        riesgos_ordenados = df_sem_ordenado["riesgo_id"].tolist()

        # Si faltan riesgos, rellenar orden con los que falten
        for r in ["R01", "R02", "R03"]:
            if r not in riesgos_ordenados:
                riesgos_ordenados.append(r)

        # Tomar solo los 3 en orden
        r_top, r_mid, r_low = riesgos_ordenados[:3]

        # Cantidades por riesgo (50% / 30% / 20%)
        n_total = occ_totales_por_semana
        n_top = int(round(n_total * 0.50))
        n_mid = int(round(n_total * 0.30))
        n_low = n_total - n_top - n_mid  # lo que falta

        plan_semana = [
            (r_top, n_top),
            (r_mid, n_mid),
            (r_low, n_low),
        ]

        # Días de esta semana en el calendario
        dias_sem = calendario[calendario["semana"] == semana]["fecha"].tolist()
        if not dias_sem:
            continue

        # ------------------------------
        # 5) Generar OCC para esta semana
        # ------------------------------
        for riesgo_id, n_occ in plan_semana:

            for _ in range(n_occ):
                # Elegir día
                fecha = random.choice(dias_sem)

                # Elegir área según mapeo OMS
                area_id = elegir_area_para_riesgo(riesgo_id)
                if area_id is None:
                    continue

                # Roles / tareas del área
                roles_area = roles_por_area.get(area_id, [])
                tareas_area = tareas_por_area.get(area_id, [])

                if not roles_area or not tareas_area:
                    continue

                # OCC+ / OCC− (por defecto 70% positivas, 30% negativas)
                estado = "OCC+" if random.random() < 0.7 else "OCC-"

                # Control crítico (si existe)
                control_id = elegir_control_critico(riesgo_id)
                is_cc = control_id is not None

                # Buscar semana/id_dia consistentes con calendario
                row_dia = calendario[calendario["fecha"] == fecha]
                if row_dia.empty:
                    continue

                semana_real = int(row_dia["semana"].iloc[0])

                registros.append({
                    "id_observacion": None,
                    "fecha": fecha,
                    "semana": semana_real,
                    "id_area": area_id,
                    "rol_observador_id": random.choice(roles_area),
                    "rol_observado_id": random.choice(roles_area),
                    "tarea_id": random.choice(tareas_area),

                    "tipo_observacion": "OCC",
                    "estado": estado,

                    "riesgo_id": riesgo_id,
                    "is_control_critico": is_cc,
                    "control_critico_id": control_id,

                    # OCC no vienen de eventos → sin vínculo a df_eventos
                    "id_evento_origen": None,
                    "tipo_evento_origen": None,
                })

    df_occ = pd.DataFrame(registros)
    return df_occ


# Ejecutar generación OCC planificadas
df_occ = generar_occ_planificadas(
    calendario,
    oms,
    roles_por_area,
    tareas_por_area,
    df_controles,
    path_proactivo=PATH_PROACTIVO,
    occ_totales_por_semana=16,   # 16 x 12 semanas ≈ 192 OCC (rango cercano a 200)
)

print("OCC generadas:", df_occ.shape)
print(df_occ["riesgo_id"].value_counts(dropna=False))
print(df_occ["estado"].value_counts(dropna=False))
df_occ.head()


OCC generadas: (192, 14)
riesgo_id
R03    68
R01    64
R02    60
Name: count, dtype: int64
estado
OCC+    126
OCC-     66
Name: count, dtype: int64


,id_observacion,fecha,semana,id_area,rol_observador_id,rol_observado_id,tarea_id,tipo_observacion,estado,riesgo_id,is_control_critico,control_critico_id,id_evento_origen,tipo_evento_origen
0,None,2024-11-12,1,MRA,MRA_ROL_08,MRA_ROL_02,MRA_TA_022,OCC,OCC+,R01,True,R01CC06,None,None
1,None,2024-11-15,1,MRA,MRA_ROL_01,MRA_ROL_06,MRA_TA_021,OCC,OCC-,R01,True,R01CC02,None,None
2,None,2024-11-12,1,STM,STM_ROL_08,STM_ROL_04,STM_TA_011,OCC,OCC-,R01,True,R01CC05,None,None
3,None,2024-11-15,1,STM,STM_ROL_04,STM_ROL_01,STM_TA_013,OCC,OCC-,R01,True,R01CM03,None,None
4,None,2024-11-13,1,STM,STM_ROL_08,STM_ROL_08,STM_TA_018,OCC,OCC+,R01,True,R01CC02,None,None


In [19]:
# ------------------------------------------------------------
# Celda 4.3 — Consolidar OPG y OCC en un solo dataframe final
# ------------------------------------------------------------

def consolidar_observaciones(df_opg, df_occ):
    """
    Une OPG y OCC en un único DF y asigna IDs únicos.
    Se mantiene completamente alineado al STDE_SCHEMA v3.
    """

    # --------------------------------------------------------
    # 1) Unir dataframes (si OCC está vacío no rompe nada)
    # --------------------------------------------------------
    df_obs = pd.concat([df_opg, df_occ], ignore_index=True)

    if df_obs.empty:
        print("⚠ No se generaron observaciones.")
        return df_obs

    # --------------------------------------------------------
    # 2) Generar ID único para cada observación
    # Formato: OBS_000001
    # --------------------------------------------------------
    df_obs["id_observacion"] = [
        f"OBS_{i:06d}" for i in range(1, len(df_obs) + 1)
    ]

    # --------------------------------------------------------
    # 3) Asegurar tipos finales para consistencia
    # --------------------------------------------------------
    df_obs["is_control_critico"] = df_obs["is_control_critico"].astype(bool)

    # --------------------------------------------------------
    # 4) Ordenar columnas según STDE_SCHEMA
    # --------------------------------------------------------
    columnas_finales = [
        "id_observacion",
        "fecha",
        "semana",
        "id_area",
        "rol_observador_id",
        "rol_observado_id",
        "tarea_id",
        "tipo_observacion",     # OPG / OCC
        "estado",               # OPG+, OPG−, OCC+, OCC−
        "riesgo_id",
        "is_control_critico",
        "control_critico_id",
    ]

    # Algunas columnas pueden faltar (ej: OCC vacío)
    columnas_presentes = [c for c in columnas_finales if c in df_obs.columns]

    df_obs = df_obs[columnas_presentes]

    # --------------------------------------------------------
    # 5) Ordenar por fecha y área (más ordenado para análisis)
    # --------------------------------------------------------
    df_obs = df_obs.sort_values(by=["fecha", "id_area"]).reset_index(drop=True)

    return df_obs


# Ejecutar consolidación
df_observaciones = consolidar_observaciones(df_opg, df_occ)

print("Observaciones totales:", df_observaciones.shape)
df_observaciones.head()


Observaciones totales: (1125, 12)


,id_observacion,fecha,semana,id_area,rol_observador_id,rol_observado_id,tarea_id,tipo_observacion,estado,riesgo_id,is_control_critico,control_critico_id
0,OBS_000004,2024-11-11,1,CHP,CHP_ROL_06,CHP_ROL_06,CHP_TA_017,OPG,OPG+,None,False,None
1,OBS_000001,2024-11-11,1,MRA,MRA_ROL_07,MRA_ROL_07,MRA_TA_016,OPG,OPG+,None,False,None
2,OBS_000002,2024-11-11,1,MRA,MRA_ROL_06,MRA_ROL_06,MRA_TA_020,OPG,OPG+,None,False,None
3,OBS_000003,2024-11-11,1,MRA,MRA_ROL_06,MRA_ROL_06,MRA_TA_012,OPG,OPG+,None,False,None
4,OBS_000005,2024-11-11,1,PLC,PLC_ROL_04,PLC_ROL_04,PLC_TA_015,OPG,OPG+,None,False,None


In [20]:
# ------------------------------------------------------------
# Celda 5.1 — Auditorías AUF reactivas (solo NMS e IMEN)
# ------------------------------------------------------------

def generar_auditorias_reactivas(df_eventos, roles_por_area):
    """
    Genera auditorías reactivas cuando existen:
      - Near Miss (NMS)
      - Incidente Menor (IMEN)

    Regla STDE v3:
      → auditoría en ≤48 horas
      → área del evento
      → riesgo focal del evento
    """

    registros = []
    id_counter = 1

    eventos_reactivos = df_eventos[
        df_eventos["tipo_evento"].isin(["NMS", "IMEN"])
    ]

    for _, ev in eventos_reactivos.iterrows():

        # Fecha AUDIT entre el mismo día y 2 días después
        fecha_ev = ev["fecha"]
        delta = random.choice([0, 1, 2])
        fecha_aud = fecha_ev + timedelta(days=delta)

        # Rol auditor aleatorio del área
        area = ev["id_area"]
        posibles_roles = roles_por_area.get(area, [])
        rol_auditor = random.choice(posibles_roles) if posibles_roles else None

        registros.append({
            "id_auditoria": f"AUF_R_{id_counter:05d}",
            "fecha": fecha_aud,
            "semana": ev["semana"],
            "id_area": ev["id_area"],
            "riesgo_focal": ev["riesgo_id"],
            "tipo_auditoria": "AUF",        # única categoría en STDE v3
            "origen": "reactiva",
            "rol_auditor_id": rol_auditor,
            "id_evento_asociado": ev.get("id_evento", None),
        })

        id_counter += 1

    return pd.DataFrame(registros)


# Ejecutar AUF REACTIVAS
df_auf_reactivas = generar_auditorias_reactivas(df_eventos, roles_por_area)

print("AUF reactivas generadas:", df_auf_reactivas.shape)
df_auf_reactivas.head()


AUF reactivas generadas: (44, 9)


,id_auditoria,fecha,semana,id_area,riesgo_focal,tipo_auditoria,origen,rol_auditor_id,id_evento_asociado
0,AUF_R_00001,2024-11-13,1,STM,R03,AUF,reactiva,STM_ROL_01,NaN
1,AUF_R_00002,2024-11-18,2,TME,R01,AUF,reactiva,TME_ROL_02,NaN
2,AUF_R_00003,2024-11-22,2,CHP,R02,AUF,reactiva,CHP_ROL_08,NaN
3,AUF_R_00004,2024-11-24,2,PLC,R01,AUF,reactiva,PLC_ROL_03,NaN
4,AUF_R_00005,2024-11-25,3,CHP,R03,AUF,reactiva,CHP_ROL_01,NaN


In [21]:
# ------------------------------------------------------------
# Celda 5.2 — Auditorías AUF planificadas (corregida)
# ------------------------------------------------------------

def generar_auditorias_planificadas(calendario, roles_por_area, riesgos=["R01","R02","R03"]):
    """
    Genera ~5 auditorías por riesgo por mes (~15 por mes total),
    distribuidas aleatoriamente dentro de cada mes.
    """

    registros = []
    id_counter = 1

    # ---------------------------------------------------------
    # 🔥 FIX: asegurar que 'fecha' sea datetime
    # ---------------------------------------------------------
    calendario["fecha"] = pd.to_datetime(calendario["fecha"])

    # Determinar meses presentes en el calendario
    calendario["mes"] = calendario["fecha"].dt.to_period("M")
    meses = calendario["mes"].unique()

    for mes in meses:
        df_mes = calendario[calendario["mes"] == mes]

        for riesgo in riesgos:
            # generar 5 auditorías planificadas para este riesgo
            for _ in range(5):

                row = df_mes.sample(1).iloc[0]
                fecha = row["fecha"]
                semana = row["semana"]

                # seleccionar área y rol auditor
                area = random.choice(list(roles_por_area.keys()))
                rol = random.choice(roles_por_area[area])

                registros.append({
                    "id_auditoria": f"AUF_P_{id_counter:05d}",
                    "fecha": fecha,
                    "semana": semana,
                    "id_area": area,
                    "riesgo_focal": riesgo,
                    "tipo_auditoria": "AUF",
                    "origen": "planificada",
                    "rol_auditor_id": rol,
                })

                id_counter += 1

    return pd.DataFrame(registros)


# Ejecutar AUF PLANIFICADAS
df_auf_planificadas = generar_auditorias_planificadas(calendario, roles_por_area)

print("AUF planificadas:", df_auf_planificadas.shape)
df_auf_planificadas.head()


AUF planificadas: (60, 8)


,id_auditoria,fecha,semana,id_area,riesgo_focal,tipo_auditoria,origen,rol_auditor_id
0,AUF_P_00001,2024-11-28,3,CHP,R01,AUF,planificada,CHP_ROL_07
1,AUF_P_00002,2024-11-16,1,TME,R01,AUF,planificada,TME_ROL_08
2,AUF_P_00003,2024-11-25,3,MRA,R01,AUF,planificada,MRA_ROL_08
3,AUF_P_00004,2024-11-14,1,STM,R01,AUF,planificada,STM_ROL_05
4,AUF_P_00005,2024-11-22,2,CHP,R01,AUF,planificada,CHP_ROL_07


In [22]:
# ------------------------------------------------------------
# Celda 5.3 — Consolidar Auditorías AUF (reactivas + planificadas)
# ------------------------------------------------------------

def consolidar_auditorias(df_auf_reactivas, df_auf_planificadas):
    """
    Une AUF reactivas y planificadas en un único dataframe,
    asigna IDs únicos finales y ordena columnas según STDE_SCHEMA v3.
    """

    # --------------------------------------------------------
    # 1) Unir ambos dataframes
    # --------------------------------------------------------
    df_aud = pd.concat(
        [df_auf_reactivas, df_auf_planificadas],
        ignore_index=True
    )

    if df_aud.empty:
        print("⚠ No se generaron auditorías.")
        return df_aud

    # --------------------------------------------------------
    # 2) Asegurar fecha como datetime (FIX del error)
    # --------------------------------------------------------
    df_aud["fecha"] = pd.to_datetime(df_aud["fecha"])

    # --------------------------------------------------------
    # 3) Asignación de ID final único
    #    Formato: AUF_000001
    # --------------------------------------------------------
    df_aud["id_auditoria_final"] = [
        f"AUF_{i:06d}" for i in range(1, len(df_aud) + 1)
    ]

    # --------------------------------------------------------
    # 4) Reordenar columnas según STDE_SCHEMA v3
    # --------------------------------------------------------
    columnas_finales = [
        "id_auditoria_final",
        "fecha",
        "semana",
        "id_area",
        "riesgo_focal",
        "tipo_auditoria",
        "origen",
        "rol_auditor_id",
        "id_evento_asociado",
    ]

    columnas_presentes = [c for c in columnas_finales if c in df_aud.columns]
    df_aud = df_aud[columnas_presentes]

    # --------------------------------------------------------
    # 5) Ordenar (ya no fallará)
    # --------------------------------------------------------
    df_aud = df_aud.sort_values(by=["fecha", "id_area"]).reset_index(drop=True)

    return df_aud


# Ejecutar consolidación final
df_auditorias = consolidar_auditorias(df_auf_reactivas, df_auf_planificadas)

print("AUDITORÍAS TOTALES:", df_auditorias.shape)
df_auditorias.head()


AUDITORÍAS TOTALES: (104, 9)


,id_auditoria_final,fecha,semana,id_area,riesgo_focal,tipo_auditoria,origen,rol_auditor_id,id_evento_asociado
0,AUF_000054,2024-11-11,1,TME,R02,AUF,planificada,TME_ROL_01,NaN
1,AUF_000052,2024-11-12,1,TME,R02,AUF,planificada,TME_ROL_04,NaN
2,AUF_000001,2024-11-13,1,STM,R03,AUF,reactiva,STM_ROL_01,NaN
3,AUF_000048,2024-11-14,1,STM,R01,AUF,planificada,STM_ROL_05,NaN
4,AUF_000046,2024-11-16,1,TME,R01,AUF,planificada,TME_ROL_08,NaN


In [29]:
# ------------------------------------------------------------
# Celda 6.1 — Cargar ranking semanal del modelo proactivo (CSV)
# ------------------------------------------------------------
# PATH_PROACTIVO ya definido en la sección 0.x, por ejemplo:
# PATH_PROACTIVO = "../data/oms/stde_proactivo_semanal_v4_4.csv"

df_proactivo = pd.read_csv(PATH_PROACTIVO)

print("Ranking proactivo cargado:", df_proactivo.shape)

# Validación básica de estructura
expected_cols = {"semana_id", "riesgo_id", "score_proactivo", "rank_proactivo"}
missing = expected_cols.difference(df_proactivo.columns)
if missing:
    raise ValueError(f"El archivo proactivo no tiene las columnas esperadas. Faltan: {missing}")

print("Columnas del proactivo:", list(df_proactivo.columns))
print("Semanas presentes:", sorted(df_proactivo["semana_id"].unique()))
print("Riesgos presentes:", sorted(df_proactivo["riesgo_id"].unique()))

df_proactivo.head()


Ranking proactivo cargado: (120, 4)
Columnas del proactivo: ['semana_id', 'riesgo_id', 'score_proactivo', 'rank_proactivo']
Semanas presentes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Riesgos presentes: ['R01', 'R02', 'R03', 'R04', 'R05', 'R06', 'R07', 'R08', 'R09', 'R10']


,semana_id,riesgo_id,score_proactivo,rank_proactivo
0,1,R01,0.91,1
1,1,R02,0.88,2
2,1,R05,0.86,3
3,1,R03,0.84,4
4,1,R04,0.82,5


In [30]:
# Celda 6.2 — Quedarnos solo con los riesgos modelados (R01–R03)

def filtrar_top3_proactivo(df):
    """
    Mantiene solo los riesgos R01–R03 del ranking semanal.
    Cada fila representa semana + riesgo + ranking_pos.
    """
    df2 = df[df["riesgo_id"].isin(["R01", "R02", "R03"])].copy()
    return df2

df_proactivo_top3 = filtrar_top3_proactivo(df_proactivo)

print("Ranking filtrado a R01–R03:", df_proactivo_top3.shape)
df_proactivo_top3.head()


Ranking filtrado a R01–R03: (36, 4)


,semana_id,riesgo_id,score_proactivo,rank_proactivo
0,1,R01,0.91,1
1,1,R02,0.88,2
3,1,R03,0.84,4
10,2,R01,0.88,1
11,2,R02,0.85,2


In [33]:
print(df_trayectorias_semana.columns)
df_trayectorias_semana.head()


Index(['semana', 'criticidad_R01_media', 'criticidad_R02_media',
       'criticidad_R03_media', 'criticidad_global_media'],
      dtype='object')


,semana,criticidad_R01_media,criticidad_R02_media,criticidad_R03_media,criticidad_global_media
0,1,0.238999,0.375859,0.175273,0.291497
1,2,0.298600,0.410016,0.184553,0.325796
2,3,0.277549,0.437501,0.193956,0.336627
3,4,0.321048,0.491794,0.170725,0.368840
4,5,0.319023,0.540638,0.175150,0.393862


In [34]:
# Celda 6.3 — Integrar trayectorias reales vs ranking semanal (R01–R03)

# df_proactivo_top3 tiene: semana_id, riesgo_id, score_proactivo, rank_proactivo
# df_trayectorias_semana tiene: semana, criticidad_R01_media, criticidad_R02_media, criticidad_R03_media, criticidad_global_media

df_compare = df_proactivo_top3.merge(
    df_trayectorias_semana,
    left_on="semana_id",
    right_on="semana",
    how="left"
)

print("Comparación ranking + trayectorias:", df_compare.shape)
df_compare.head()


Comparación ranking + trayectorias: (36, 9)


,semana_id,riesgo_id,score_proactivo,rank_proactivo,semana,criticidad_R01_media,criticidad_R02_media,criticidad_R03_media,criticidad_global_media
0,1,R01,0.91,1,1,0.238999,0.375859,0.175273,0.291497
1,1,R02,0.88,2,1,0.238999,0.375859,0.175273,0.291497
2,1,R03,0.84,4,1,0.238999,0.375859,0.175273,0.291497
3,2,R01,0.88,1,2,0.298600,0.410016,0.184553,0.325796
4,2,R02,0.85,2,2,0.298600,0.410016,0.184553,0.325796


In [36]:
# Celda 6.4 — Clasificación K9 basada en criticidad y tendencia

def clasificacion_k9(criticidad_actual, criticidad_prev):
    """
    Solo 2 colores:
    - ROJO     = alta criticidad + tendencia fuerte
    - AMARILLO = moderado + tendencia leve
    """
    tendencia = criticidad_actual - criticidad_prev if criticidad_prev is not None else 0.0

    if criticidad_actual > 0.70 and tendencia > 0.05:
        return "ROJO"

    if criticidad_actual > 0.50 or tendencia > 0.02:
        return "AMARILLO"

    return "AMARILLO"  # Por diseño K9 (no usamos VERDE)


# ---------------------------------------------------
# Aplicar clasificación por riesgo / semana
# ---------------------------------------------------
clasificaciones = []

for _, row in df_compare.iterrows():
    riesgo = row["riesgo_id"]
    semana = row["semana"]

    crit_col = f"criticidad_{riesgo}_media"
    criticidad_actual = row.get(crit_col, None)

    # Criticidad semana anterior
    if semana > 1:
        prev = df_compare[
            (df_compare["riesgo_id"] == riesgo) &
            (df_compare["semana"] == semana - 1)
        ]
        # *** FIX DEL FUTUREWARNING ***
        criticidad_prev = float(prev[crit_col].iloc[0]) if len(prev) > 0 else None
    else:
        criticidad_prev = None

    color = clasificacion_k9(criticidad_actual, criticidad_prev)
    clasificaciones.append(color)

df_compare["k9_color"] = clasificaciones

print("Clasificación aplicada.")
df_compare.head()


Clasificación aplicada.


,semana_id,riesgo_id,score_proactivo,rank_proactivo,semana,criticidad_R01_media,criticidad_R02_media,criticidad_R03_media,criticidad_global_media,k9_color
0,1,R01,0.91,1,1,0.238999,0.375859,0.175273,0.291497,AMARILLO
1,1,R02,0.88,2,1,0.238999,0.375859,0.175273,0.291497,AMARILLO
2,1,R03,0.84,4,1,0.238999,0.375859,0.175273,0.291497,AMARILLO
3,2,R01,0.88,1,2,0.298600,0.410016,0.184553,0.325796,AMARILLO
4,2,R02,0.85,2,2,0.298600,0.410016,0.184553,0.325796,AMARILLO


In [38]:
# Celda 6.5 — Output final del bloque 6.x

df_resultado_6x = df_compare[[
    "semana",
    "riesgo_id",
    "rank_proactivo",
    "criticidad_R01_media",
    "criticidad_R02_media",
    "criticidad_R03_media",
    "criticidad_global_media",
    "k9_color"
]].sort_values(["semana", "rank_proactivo"])

print("Resultado final 6.x:", df_resultado_6x.shape)
df_resultado_6x.head(10)


Resultado final 6.x: (36, 8)


,semana,riesgo_id,rank_proactivo,criticidad_R01_media,criticidad_R02_media,criticidad_R03_media,criticidad_global_media,k9_color
0,1,R01,1,0.238999,0.375859,0.175273,0.291497,AMARILLO
1,1,R02,2,0.238999,0.375859,0.175273,0.291497,AMARILLO
2,1,R03,4,0.238999,0.375859,0.175273,0.291497,AMARILLO
3,2,R01,1,0.298600,0.410016,0.184553,0.325796,AMARILLO
4,2,R02,2,0.298600,0.410016,0.184553,0.325796,AMARILLO
5,2,R03,5,0.298600,0.410016,0.184553,0.325796,AMARILLO
6,3,R01,1,0.277549,0.437501,0.193956,0.336627,AMARILLO
7,3,R02,3,0.277549,0.437501,0.193956,0.336627,AMARILLO
8,3,R03,4,0.277549,0.437501,0.193956,0.336627,AMARILLO
9,4,R03,1,0.321048,0.491794,0.170725,0.368840,AMARILLO


In [39]:
# Celda 6.6 — Señales individuales por riesgo

def calcular_senales_individuales(df_compare):

    registros = []

    for semana in sorted(df_compare["semana"].unique()):
        df_sem = df_compare[df_compare["semana"] == semana]

        # Ordenar riesgos por rank_proactivo (corrección)
        df_sem_sorted = df_sem.sort_values("rank_proactivo")

        # Obtener criticidad del top1 para la señal “distancia”
        riesgo_top1 = df_sem_sorted.iloc[0]["riesgo_id"]
        crit_top1 = df_sem_sorted.iloc[0][f"criticidad_{riesgo_top1}_media"]

        for _, row in df_sem_sorted.iterrows():
            riesgo = row["riesgo_id"]

            crit_actual = row[f"criticidad_{riesgo}_media"]

            # Criticidad previa
            if semana > 1:
                df_prev = df_compare[
                    (df_compare["riesgo_id"] == riesgo) &
                    (df_compare["semana"] == semana - 1)
                ]
                # FIX: float() sobre Series → usamos iloc[0]
                crit_prev = float(df_prev[f"criticidad_{riesgo}_media"].iloc[0]) if len(df_prev) > 0 else None
            else:
                crit_prev = None

            tendencia = (crit_actual - crit_prev) if crit_prev is not None else 0.0

            # Severidad interna K9 (SEV1/2/3)
            if row["k9_color"] == "ROJO":
                sev = "SEV3"
            else:
                sev = "SEV2" if tendencia > 0.03 else "SEV1"

            # Distancia al top 1
            dist_top1 = crit_actual - crit_top1

            # Shock del lunes crítico (si existiera semana 13)
            lc_shock = 1 if semana == 13 and riesgo in ["R01", "R02"] else 0
            lc_delta = tendencia if lc_shock == 1 else 0.0

            registros.append({
                "semana": semana,
                "riesgo_id": riesgo,
                "rank_proactivo": row["rank_proactivo"],  # corrección de ranking_pos

                # Señales STDE
                "criticidad": crit_actual,
                "tendencia": tendencia,
                "dist_top1": dist_top1,

                # Señal interna K9
                "sev_k9": sev,

                # Señales lunes crítico
                "lc_shock": lc_shock,
                "lc_delta": lc_delta,
            })

    return pd.DataFrame(registros)


df_senales = calcular_senales_individuales(df_compare)

print("Señales individuales generadas:", df_senales.shape)
df_senales.head(10)


Señales individuales generadas: (36, 9)


,semana,riesgo_id,rank_proactivo,criticidad,tendencia,dist_top1,sev_k9,lc_shock,lc_delta
0,1,R01,1,0.238999,0.000000,0.000000,SEV1,0,0.0
1,1,R02,2,0.375859,0.000000,0.136860,SEV1,0,0.0
2,1,R03,4,0.175273,0.000000,-0.063726,SEV1,0,0.0
3,2,R01,1,0.298600,0.059601,0.000000,SEV2,0,0.0
4,2,R02,2,0.410016,0.034157,0.111416,SEV2,0,0.0
5,2,R03,5,0.184553,0.009280,-0.114047,SEV1,0,0.0
6,3,R01,1,0.277549,-0.021051,0.000000,SEV1,0,0.0
7,3,R02,3,0.437501,0.027485,0.159952,SEV1,0,0.0
8,3,R03,4,0.193956,0.009403,-0.083593,SEV1,0,0.0
9,4,R03,1,0.170725,-0.023231,0.000000,SEV1,0,0.0


In [43]:
# ------------------------------------------------------------
# Celda 7.1 – Extraer señales del lunes crítico (versión corregida)
# ------------------------------------------------------------

def extraer_senales_lunes_critico(df_eventos, df_tray_sem):
    """
    Extrae la señal del lunes crítico como APORTACIÓN DE EVIDENCIA,
    no como comparación contra la semana previa.
    
    - criticidad_previa: estado acumulado hasta semana 12
    - criticidad_lc: criticidad del evento LC
    - delta: aporte de riesgo del LC (criticidad_lc * factor_FDO)

    Reglas:
      * Los eventos SIEMPRE SUMAN, no restan.
      * R01 tiene factor 1.5 porque su LC era visible en FDO/Trayectorias.
      * R02 tiene factor 1.0 porque ya era dominante según el proactivo.
    """

    df_lc = df_eventos[df_eventos["es_lunes_critico"] == True].copy()

    senales = {}

    for riesgo in ["R01", "R02"]:
        df_r = df_lc[df_lc["riesgo_id"] == riesgo]
        if df_r.empty:
            continue

        # criticidad del evento lunes crítico
        criticidad_lc = float(df_r["criticidad"].iloc[0])

        # criticidad acumulada (semana 12)
        crit_prev = float(
            df_tray_sem[df_tray_sem["semana"] == 12][f"criticidad_{riesgo}_media"].iloc[0]
        )

        # FACTOR FDO
        if riesgo == "R01":
            factor_fdo = 1.5   # pseudo-outlier detectado por FDO
        else:
            factor_fdo = 1.0   # riesgo ya dominante para el proactivo

        # Nueva definición de delta: APORTACIÓN, no comparación
        delta = round(criticidad_lc * factor_fdo, 4)

        senales[riesgo] = {
            "criticidad_previa": round(crit_prev, 4),
            "criticidad_lc": round(criticidad_lc, 4),
            "delta": delta
        }

    return senales


# usar df_trayectorias_semana (tu variable real del notebook)
senales_lc = extraer_senales_lunes_critico(df_eventos, df_trayectorias_semana)

print("Señales del lunes crítico extraídas:")
senales_lc


Señales del lunes crítico extraídas:


{'R01': {'criticidad_previa': 0.5044,
  'criticidad_lc': 0.4795,
  'delta': 0.7192},
 'R02': {'criticidad_previa': 0.8593,
  'criticidad_lc': 0.8867,
  'delta': 0.8867}}

In [44]:
# ------------------------------------------------------------
# Celda 7.2 – Recalibración conceptual del modelo proactivo (corregida)
# ------------------------------------------------------------

def recalibrar_proactivo(df_proactivo, senales_lc):
    """
    K9 recalibra su interpretación del modelo proactivo posterior al LC.

    No modifica el score del modelo proactivo.
    Usa el delta como APORTACIÓN DE RIESGO (no comparación).
    
    Reglas:
        peso_k9 = 1 + (delta * 1.5)  si delta > 0
        peso_k9 = 1                  si delta = 0
    """

    df = df_proactivo.copy()
    df["peso_k9"] = 1.0

    for riesgo, datos in senales_lc.items():
        delta = datos["delta"]          # ahora delta siempre es aporte >= 0
        if delta > 0:
            incremento = 1 + (1.5 * delta)
            df.loc[df["riesgo_id"] == riesgo, "peso_k9"] = round(incremento,4)

    return df


df_proactivo_recal = recalibrar_proactivo(df_proactivo_thresholds, senales_lc)

print("Recalibración conceptual aplicada (peso_k9):")
df_proactivo_recal.head(10)


Recalibración conceptual aplicada (peso_k9):


,semana_id,riesgo_id,score_proactivo,rank_proactivo,threshold_alerta,threshold_critico,peso_k9
0,1,R01,0.91,1,0.83,0.86,2.0788
1,1,R02,0.88,2,0.83,0.86,2.3300
2,1,R05,0.86,3,0.83,0.86,1.0000
3,1,R03,0.84,4,0.83,0.86,1.0000
4,1,R04,0.82,5,0.83,0.86,1.0000
5,1,R07,0.80,6,0.83,0.86,1.0000
6,1,R06,0.78,7,0.83,0.86,1.0000
7,1,R08,0.76,8,0.83,0.86,1.0000
8,1,R10,0.74,9,0.83,0.86,1.0000
9,1,R09,0.72,10,0.83,0.86,1.0000


In [45]:
# ------------------------------------------------------------
# Celda 7.3 – Señales recalibradas finales para AnalystNode K9 (corregida)
# ------------------------------------------------------------

def generar_senales_k9(df_tray_sem, df_proactivo_recal):
    """
    Genera para cada semana y riesgo (R01-R03):

    - rank_pos
    - score (score_proactivo)
    - dist_top1
    - peso_k9
    - criticidad_media
    - tendencia
    - sev_k9 (SEV1–SEV3)
    """

    registros = []
    semanas = sorted(df_tray_sem["semana"].unique())

    for semana in semanas:
        for riesgo in ["R01", "R02", "R03"]:

            # criticidad media
            crit = float(
                df_tray_sem[df_tray_sem["semana"] == semana][f"criticidad_{riesgo}_media"].iloc[0]
            )

            # tendencia
            if semana == 1:
                tendencia = 0.0
            else:
                crit_prev = float(
                    df_tray_sem[df_tray_sem["semana"] == semana - 1][f"criticidad_{riesgo}_media"].iloc[0]
                )
                tendencia = crit - crit_prev

            # ranking del proactivo + peso K9
            df_rk = df_proactivo_recal[
                (df_proactivo_recal["semana_id"] == semana) &
                (df_proactivo_recal["riesgo_id"] == riesgo)
            ]

            if df_rk.empty:
                continue

            rank_pos = int(df_rk["rank_proactivo"].iloc[0])
            score = float(df_rk["score_proactivo"].iloc[0])
            peso_k9 = float(df_rk["peso_k9"].iloc[0])

            # distancia al top 1
            score_top1 = float(
                df_proactivo_recal[df_proactivo_recal["semana_id"] == semana]
                .sort_values("rank_proactivo")
                .iloc[0]["score_proactivo"]
            )
            dist_top1 = score - score_top1

            # severidad K9 basada en score proactivo
            if score >= 0.8:
                sev = "SEV3"
            elif score >= 0.5:
                sev = "SEV2"
            else:
                sev = "SEV1"

            registros.append({
                "semana": semana,
                "riesgo_id": riesgo,
                "criticidad_media": round(crit, 4),
                "tendencia": round(tendencia, 4),
                "rank_pos": rank_pos,
                "score": round(score, 4),
                "dist_top1": round(dist_top1, 4),
                "sev_k9": sev,
                "peso_k9": round(peso_k9, 4),
            })

    return pd.DataFrame(registros)


# FIX: variable correcta = df_trayectorias_semana
df_senales_k9 = generar_senales_k9(df_trayectorias_semana, df_proactivo_recal)

print("Señales K9 generadas:", df_senales_k9.shape)
df_senales_k9.head(15)


Señales K9 generadas: (36, 9)


,semana,riesgo_id,criticidad_media,tendencia,rank_pos,score,dist_top1,sev_k9,peso_k9
0,1,R01,0.2390,0.0000,1,0.91,0.00,SEV3,2.0788
1,1,R02,0.3759,0.0000,2,0.88,-0.03,SEV3,2.3300
2,1,R03,0.1753,0.0000,4,0.84,-0.07,SEV3,1.0000
3,2,R01,0.2986,0.0596,1,0.88,0.00,SEV3,2.0788
4,2,R02,0.4100,0.0342,2,0.85,-0.03,SEV3,2.3300
5,2,R03,0.1846,0.0093,5,0.79,-0.09,SEV2,1.0000
6,3,R01,0.2775,-0.0211,1,0.90,0.00,SEV3,2.0788
7,3,R02,0.4375,0.0275,3,0.85,-0.05,SEV3,2.3300
8,3,R03,0.1940,0.0094,4,0.83,-0.07,SEV3,1.0000
9,4,R01,0.3210,0.0435,3,0.82,-0.05,SEV3,2.0788


In [48]:
# ------------------------------------------------------------
# Celda 8.1 – Exportación de datasets STDE v3 + FDO a CSV
# ------------------------------------------------------------

import os

OUTPUT_DIR = "../data/synthetic/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- EXPORTACIONES BASE DEL STDE v3 ---
files_to_export = {
    "stde_trayectorias_diarias.csv": df_trayectorias,
    "stde_trayectorias_semanales.csv": df_trayectorias_semana,
    "stde_eventos.csv": df_eventos,
    "stde_observaciones.csv": df_observaciones,
    "stde_auditorias.csv": df_auditorias,
    "stde_proactivo_semanal_v4_4_thresholds.csv": df_proactivo_thresholds,
    "stde_senales_k9.csv": df_senales_k9,
}

# Exportación datasets base
for filename, df in files_to_export.items():
    path = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(path, index=False, encoding="utf-8")
    print(f"✓ Exportado: {filename} → {path}")

# --- EXPORTACIÓN FDO DIARIO COMPLETO ---
fdo_full_path = os.path.join(OUTPUT_DIR, "stde_fdo_diario.csv")
df_fdo_diario.to_csv(fdo_full_path, index=False, encoding="utf-8")
print(f"✓ Exportado: stde_fdo_diario.csv → {fdo_full_path}")

# --- EXPORTACIÓN SEÑALES FDO POR SEPARADO ---
fdo_columns = [
    "fdo_produccion",
    "fdo_backlog",
    "fdo_congestion",
    "fdo_fatiga",
    "fdo_clima",
    "fdo_dotacion",
    "fdo_variabilidad",
]

for col in fdo_columns:
    df_tmp = df_fdo_diario[["id_dia", "fecha", "semana", col]].copy()
    filename = f"stde_{col}.csv"
    path = os.path.join(OUTPUT_DIR, filename)
    df_tmp.to_csv(path, index=False, encoding="utf-8")
    print(f"✓ Exportado: {filename} → {path}")

print("\nExportación STDE v3 + FDO completada.")


✓ Exportado: stde_trayectorias_diarias.csv → ../data/synthetic/stde_trayectorias_diarias.csv
✓ Exportado: stde_trayectorias_semanales.csv → ../data/synthetic/stde_trayectorias_semanales.csv
✓ Exportado: stde_eventos.csv → ../data/synthetic/stde_eventos.csv
✓ Exportado: stde_observaciones.csv → ../data/synthetic/stde_observaciones.csv
✓ Exportado: stde_auditorias.csv → ../data/synthetic/stde_auditorias.csv
✓ Exportado: stde_proactivo_semanal_v4_4_thresholds.csv → ../data/synthetic/stde_proactivo_semanal_v4_4_thresholds.csv
✓ Exportado: stde_senales_k9.csv → ../data/synthetic/stde_senales_k9.csv
✓ Exportado: stde_fdo_diario.csv → ../data/synthetic/stde_fdo_diario.csv
✓ Exportado: stde_fdo_produccion.csv → ../data/synthetic/stde_fdo_produccion.csv
✓ Exportado: stde_fdo_backlog.csv → ../data/synthetic/stde_fdo_backlog.csv
✓ Exportado: stde_fdo_congestion.csv → ../data/synthetic/stde_fdo_congestion.csv
✓ Exportado: stde_fdo_fatiga.csv → ../data/synthetic/stde_fdo_fatiga.csv
✓ Exportado: std

In [49]:
# ------------------------------------------------------------
# Celda 8.2 – Catálogo oficial FDO (para K9 y datasets)
# ------------------------------------------------------------

import pandas as pd

fdo_catalogo = pd.DataFrame([
    {"fdo_id": "fdo_produccion", "descripcion": "Variabilidad en el desempeño productivo diario, afecta cargas operativas y presión por resultados.", "tipo": "operacional"},
    {"fdo_id": "fdo_backlog", "descripcion": "Acumulación de tareas pendientes que incrementa la presión operativa y aumenta condiciones subóptimas.", "tipo": "organizacional"},
    {"fdo_id": "fdo_congestion", "descripcion": "Nivel de interferencias operativas y congestión de equipos/áreas.", "tipo": "operacional"},
    {"fdo_id": "fdo_fatiga", "descripcion": "Fatiga operacional derivada de condiciones de trabajo, turnos, intensidad y clima.", "tipo": "humano"},
    {"fdo_id": "fdo_clima", "descripcion": "Factores climáticos que degradan la estabilidad operacional o aumentan el riesgo.", "tipo": "entorno"},
    {"fdo_id": "fdo_dotacion", "descripcion": "Cambios o restricciones en la dotación disponible, afecta supervisión y capacidad de control.", "tipo": "organizacional"},
    {"fdo_id": "fdo_variabilidad", "descripcion": "Variabilidad global del sistema ante perturbaciones no anticipadas.", "tipo": "sistema"},
])

catalog_path = os.path.join(OUTPUT_DIR, "stde_fdo_catalogo.csv")
fdo_catalogo.to_csv(catalog_path, index=False, encoding="utf-8")

print("✓ Exportado catálogo FDO →", catalog_path)
fdo_catalogo


✓ Exportado catálogo FDO → ../data/synthetic/stde_fdo_catalogo.csv


,fdo_id,descripcion,tipo
0,fdo_produccion,Variabilidad en el desempeño productivo diario...,operacional
1,fdo_backlog,Acumulación de tareas pendientes que increment...,organizacional
2,fdo_congestion,Nivel de interferencias operativas y congestió...,operacional
3,fdo_fatiga,Fatiga operacional derivada de condiciones de ...,humano
4,fdo_clima,Factores climáticos que degradan la estabilida...,entorno
5,fdo_dotacion,Cambios o restricciones en la dotación disponi...,organizacional
6,fdo_variabilidad,Variabilidad global del sistema ante perturbac...,sistema


In [57]:
# ------------------------------------------------------------
# Celda 8.3 – Checks de consistencia STDE v3 (versión FINAL corregida)
# ------------------------------------------------------------

print("------------------------------------------------------------")
print("CHECKS DE CONSISTENCIA – STDE v3")
print("------------------------------------------------------------")

errores = []

# ============================================================
# 1) Validación de riesgos según ontología
# Ontología usa df_riesgos["id"]
# ============================================================

riesgos_validos = set(df_riesgos["id"].unique())

riesgos_eventos = set(df_eventos["riesgo_id"].dropna().unique())
riesgos_observ = set(df_observaciones["riesgo_id"].dropna().unique())

if not riesgos_eventos.issubset(riesgos_validos):
    errores.append("❌ Riesgos desconocidos en eventos.")
else:
    print("✓ Riesgos válidos en eventos.")

if not riesgos_observ.issubset(riesgos_validos):
    errores.append("❌ Riesgos desconocidos en observaciones.")
else:
    print("✓ Riesgos válidos en observaciones.")


# ============================================================
# 2) Validación de Áreas Operacionales
# Ontología usa df_areas["id"]
# Eventos usan df_eventos["id_area"]
# ============================================================

areas_validas = set(df_areas["id"].unique())

areas_eventos = set(df_eventos["id_area"].dropna().unique())
areas_observ = set(df_observaciones["id_area"].dropna().unique())

if not areas_eventos.issubset(areas_validas):
    errores.append("❌ Áreas desconocidas en eventos.")
else:
    print("✓ Áreas válidas en eventos.")

if not areas_observ.issubset(areas_validas):
    errores.append("❌ Áreas desconocidas en observaciones.")
else:
    print("✓ Áreas válidas en observaciones.")


# ============================================================
# 3) Validación de Roles y Tareas
# Ontología usa df_roles["id"] y df_tareas["id"]
# Eventos usan rol_id / tarea_id
# ============================================================

roles_validos = set(df_roles["id"].unique())
tareas_validas = set(df_tareas["id"].unique())

roles_eventos = set(df_eventos["rol_id"].dropna().unique())
tareas_eventos = set(df_eventos["tarea_id"].dropna().unique())

if not roles_eventos.issubset(roles_validos):
    errores.append("❌ Roles desconocidos en eventos.")
else:
    print("✓ Roles válidos en eventos.")

if not tareas_eventos.issubset(tareas_validas):
    errores.append("❌ Tareas desconocidas en eventos.")
else:
    print("✓ Tareas válidas en eventos.")


# ============================================================
# 4) Validación del lunes crítico (3 registros exactos)
# ============================================================

df_lc = df_eventos[df_eventos["es_lunes_critico"] == True]

if df_lc.shape[0] != 3:
    errores.append(f"❌ Lunes crítico incorrecto: {df_lc.shape[0]} registros.")
else:
    print("✓ Lunes crítico correcto (3 registros).")


# ============================================================
# 5) Validación OPG/OCC
# ============================================================

if "tipo_observacion" in df_observaciones.columns:
    pct_opg = (df_observaciones["tipo_observacion"] == "OPG").mean()
    pct_occ = (df_observaciones["tipo_observacion"] == "OCC").mean()

    print(f"Proporción OPG: {pct_opg:.2f} | OCC: {pct_occ:.2f}")

    if pct_occ > 0.40:
        errores.append("❌ OCC demasiado altas (esperado 10–25%).")
else:
    print("⚠ df_observaciones no tiene columna 'tipo_observacion'.")


# ============================================================
# 6) Validación AUF vs carga operacional
# ============================================================

n_nms = (df_eventos["tipo_evento"] == "NMS").sum()
n_auf = df_auditorias.shape[0]

if n_auf < n_nms:
    errores.append("❌ AUF insuficientes frente a NMS.")
else:
    print("✓ AUF coherentes con carga operacional.")


# ============================================================
# 7) Validación FDO (nuevos en STDE v3)
# ============================================================

required_fdo_cols = {
    "fdo_produccion",
    "fdo_backlog",
    "fdo_congestion",
    "fdo_fatiga",
    "fdo_clima",
    "fdo_dotacion",
    "fdo_variabilidad",
}

# Convertimos set → lista (corrección Pandas)
fdo_cols = list(required_fdo_cols)

# Existencia
missing_fdo = set(fdo_cols).difference(df_fdo_diario.columns)
if missing_fdo:
    errores.append(f"❌ Señales FDO faltantes: {missing_fdo}")
else:
    print("✓ Señales FDO completas.")

# Rango
if (df_fdo_diario[fdo_cols] < 0).any().any() or \
   (df_fdo_diario[fdo_cols] > 1).any().any():
    errores.append("❌ Señales FDO fuera del rango [0–1].")
else:
    print("✓ FDO dentro del rango [0–1].")

# Nulos
if df_fdo_diario[fdo_cols].isnull().any().any():
    errores.append("❌ Señales FDO contienen nulos.")
else:
    print("✓ FDO sin nulos.")


# ============================================================
# 8) RESULTADO FINAL
# ============================================================

print("------------------------------------------------------------")

if errores:
    print("❌ Se detectaron problemas:")
    for e in errores:
        print("   -", e)
else:
    print("✓ Todos los checks STDE v3 se cumplen correctamente.")

print("------------------------------------------------------------")


------------------------------------------------------------
CHECKS DE CONSISTENCIA – STDE v3
------------------------------------------------------------
✓ Riesgos válidos en eventos.
✓ Riesgos válidos en observaciones.
✓ Áreas válidas en eventos.
✓ Áreas válidas en observaciones.
✓ Roles válidos en eventos.
✓ Tareas válidas en eventos.
✓ Lunes crítico correcto (3 registros).
Proporción OPG: 0.83 | OCC: 0.17
✓ AUF coherentes con carga operacional.
✓ Señales FDO completas.
✓ FDO dentro del rango [0–1].
✓ FDO sin nulos.
------------------------------------------------------------
✓ Todos los checks STDE v3 se cumplen correctamente.
------------------------------------------------------------


In [58]:
# ------------------------------------------------------------
# Celda 8.4 — DataFrame semanal unificado: k9_weekly_signals.parquet
# ------------------------------------------------------------

import pyarrow as pa
import pyarrow.parquet as pq

# ------------------------------------------------------------
# 1) Partimos del df_senales_k9 (ya generado en 7.x)
# ------------------------------------------------------------

df_weekly = df_senales_k9.copy()

# ------------------------------------------------------------
# 2) Agregar columnas indicadoras para AnalystNode
# ------------------------------------------------------------

# Top3 según ranking del modelo proactivo (columna rank_pos)
df_weekly["is_top3"] = df_weekly["rank_pos"].apply(lambda r: r <= 3)

# Riesgo dominante por criticidad media
df_weekly["riesgo_dominante"] = df_weekly.groupby("semana")["criticidad_media"] \
                                          .transform(lambda x: x == x.max())

# Semana del lunes crítico
semana_lc = int(df_eventos[df_eventos["es_lunes_critico"] == True]["semana"].iloc[0])
df_weekly["es_semana_lunes_critico"] = df_weekly["semana"] == semana_lc

# ------------------------------------------------------------
# 3) Exportar como parquet
# ------------------------------------------------------------

path_parquet = os.path.join(OUTPUT_DIR, "k9_weekly_signals.parquet")

table = pa.Table.from_pandas(df_weekly)
pq.write_table(table, path_parquet)

print(f"✓ Archivo semanal unificado exportado: {path_parquet}")
df_weekly.head()


✓ Archivo semanal unificado exportado: ../data/synthetic/k9_weekly_signals.parquet


,semana,riesgo_id,criticidad_media,tendencia,rank_pos,score,dist_top1,sev_k9,peso_k9,is_top3,riesgo_dominante,es_semana_lunes_critico
0,1,R01,0.2390,0.0000,1,0.91,0.00,SEV3,2.0788,True,False,False
1,1,R02,0.3759,0.0000,2,0.88,-0.03,SEV3,2.3300,True,True,False
2,1,R03,0.1753,0.0000,4,0.84,-0.07,SEV3,1.0000,False,False,False
3,2,R01,0.2986,0.0596,1,0.88,0.00,SEV3,2.0788,True,False,False
4,2,R02,0.4100,0.0342,2,0.85,-0.03,SEV3,2.3300,True,True,False


In [60]:
# ------------------------------------------------------------
# Celda 8.5 — Context Bundle JSON para AnalystNode + PredictorNode (FIX fechas)
# ------------------------------------------------------------

import json
import os

# Helper seguro para fechas
def safe_date(x):
    return str(x) if not isinstance(x, str) else x


# ================
# 1) AnalystNode payload
# ================

def build_week_payload(semana, df_weekly, df_eventos, df_observaciones, df_auditorias):

    df_w = df_weekly[df_weekly["semana"] == semana]

    eventos_w = df_eventos[df_eventos["semana"] == semana].copy()
    observ_w = df_observaciones[df_observaciones["semana"] == semana].copy()
    auf_w = df_auditorias[df_auditorias["semana"] == semana].copy()

    # Serializar fechas a string
    for df_tmp in [eventos_w, observ_w, auf_w]:
        if "fecha" in df_tmp.columns:
            df_tmp["fecha"] = df_tmp["fecha"].apply(safe_date)

    payload = {
        "semana": int(semana),
        "es_semana_lunes_critico": bool(df_w["es_semana_lunes_critico"].any()),
        "riesgos": [],
        "eventos": eventos_w.to_dict(orient="records"),
        "observaciones": observ_w.to_dict(orient="records"),
        "auditorias": auf_w.to_dict(orient="records"),
    }

    for _, row in df_w.iterrows():
        payload["riesgos"].append({
            "riesgo_id": row["riesgo_id"],
            "criticidad_media": float(row["criticidad_media"]),
            "tendencia": float(row["tendencia"]),
            "rank_pos": int(row["rank_pos"]),
            "score": float(row["score"]),
            "dist_top1": float(row["dist_top1"]),
            "sev_k9": row["sev_k9"],
            "peso_k9": float(row["peso_k9"]),
            "is_top3": bool(row["is_top3"]),
            "riesgo_dominante": bool(row["riesgo_dominante"]),
        })

    return payload


analyst_payloads = {}
for semana in sorted(df_weekly["semana"].unique()):
    analyst_payloads[f"semana_{int(semana)}"] = build_week_payload(
        semana,
        df_weekly,
        df_eventos,
        df_observaciones,
        df_auditorias
    )


# ================
# 2) PredictorNode payload
# ================

predictor_payload = {
    "descripcion": "STDE v3 – Señales completas para PredictorNode",
    "periodo": {
        "inicio": safe_date(calendario["fecha"].iloc[0]),
        "fin": safe_date(calendario["fecha"].iloc[-1]),
        "semanas": int(calendario["semana"].max())
    },

    "riesgos_modelados": ["R01", "R02", "R03"],
    "trayectorias": df_trayectorias_semana.to_dict(orient="records"),

    "eventos": {
        "totales": int(df_eventos.shape[0]),
        "por_semana": df_eventos.groupby("semana").size().to_dict(),
        "por_riesgo": df_eventos["riesgo_id"].value_counts().to_dict(),
        "near_miss": int((df_eventos["tipo_evento"] == "NMS").sum()),
        "incident_menor": int((df_eventos["tipo_evento"] == "IMEN").sum()),
        "lunes_critico": df_eventos[df_eventos["es_lunes_critico"] == True]
                             .assign(fecha=lambda d: d["fecha"].apply(safe_date))
                             .to_dict(orient="records")
    },

    "observaciones": {
        "totales": int(df_observaciones.shape[0]),
        "opg": int((df_observaciones["tipo_observacion"] == "OPG").sum()),
        "occ": int((df_observaciones["tipo_observacion"] == "OCC").sum()),
        "occ_negativas": int((df_observaciones["estado"] == "OCC-").sum())
    },

    "auditorias": {
        "totales": int(df_auditorias.shape[0]),
        "por_semana": df_auditorias.groupby("semana").size().to_dict(),
        "reactivas": int((df_auditorias["tipo_auditoria"] == "AUF").sum()),
        "planificadas": int((df_auditorias["tipo_auditoria"] == "PLANIFICADA").sum())
                        if "PLANIFICADA" in df_auditorias["tipo_auditoria"].unique()
                        else 0
    },

    "modelo_proactivo": {
        "archivo_fuente": "stde_proactivo_semanal_v4_4.csv",
        "ranking": df_proactivo_thresholds.to_dict(orient="records")
    },

    "k9_signals": {
        "por_semana": df_senales_k9.to_dict(orient="records"),
        "ruta_parquet": "k9_weekly_signals.parquet"
    },

    "fdo": df_fdo_diario.assign(
        fecha=lambda d: d["fecha"].apply(safe_date)
    ).to_dict(orient="records")
}


# ================
# 3) Context bundle final
# ================

context_bundle = {
    "metadata": {
        "version": "STDE_v3",
        "fuente": "Synthetic Engine v3",
        "periodo": f"{safe_date(calendario['fecha'].iloc[0])} → {safe_date(calendario['fecha'].iloc[-1])}"
    },
    "analystnode": analyst_payloads,
    "predictornode": predictor_payload
}

# Guardar
bundle_path = os.path.join(OUTPUT_DIR, "k9_context_bundle.json")
with open(bundle_path, "w", encoding="utf-8") as f:
    json.dump(context_bundle, f, ensure_ascii=False, indent=2)

print(f"✓ Context bundle exportado: {bundle_path}")


✓ Context bundle exportado: ../data/synthetic/k9_context_bundle.json
